# [1070. Product Sales Analysis III](https://leetcode.com/problems/product-sales-analysis-iii/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Sales

<pre>+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+</pre>
(sale_id, year) is the primary key (combination of columns with unique values) of this table.
product_id is a foreign key (reference column) to Product table.
Each row of this table shows a sale on the product product_id in a certain year.
Note that the price is per unit.


Table: Product

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| product_id   | int     |
| product_name | varchar |
+--------------+---------+</pre>
product_id is the primary key (column with unique values) of this table.
Each row of this table indicates the product name of each product.


Write a solution to select the product id, year, quantity, and price for the first year of every product sold.

Return the resulting table in any order.

The result format is in the following example.



Example 1:

Input:
Sales table:
<pre>+---------+------------+------+----------+-------+
| sale_id | product_id | year | quantity | price |
+---------+------------+------+----------+-------+
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |
+---------+------------+------+----------+-------+</pre>
Product table:
<pre>+------------+--------------+
| product_id | product_name |
+------------+--------------+
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |
+------------+--------------+</pre>
Output:
<pre>+------------+------------+----------+-------+
| product_id | first_year | quantity | price |
+------------+------------+----------+-------+
| 100        | 2008       | 10       | 5000  |
| 200        | 2011       | 15       | 9000  |
+------------+------------+----------+-------+</pre>

In [1]:
# pandas schema

import pandas as pd

data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype(
    {'sale_id': 'Int64', 'product_id': 'Int64', 'year': 'Int64', 'quantity': 'Int64', 'price': 'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
products = pd.DataFrame(data, columns=['product_id', 'product_name']).astype(
    {'product_id': 'Int64', 'product_name': 'object'})

#converting to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

sales_df = spark.createDataFrame(sales)
product_df = spark.createDataFrame(products)

In [2]:
sales

,sale_id,product_id,year,quantity,price
0,1,100,2008,10,5000
1,2,100,2009,12,5000
2,7,200,2011,15,9000


In [3]:

products

,product_id,product_name
0,100,Nokia
1,200,Apple
2,300,Samsung


In [24]:
from pyspark.sql import Window

# solving in spark dataframe
sdf=sales_df.groupBy('product_id').agg(min('year').alias('my'))

product_df.join(sdf,['product_id'],'inner')\
          .join(sales_df,['product_id'],'inner')\
          .where("year==my")\
          .select('product_id',col('year').alias('first_year'),'quantity','price')\
          .show()


+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



In [25]:
# solving spark sql

sales_df.createOrReplaceTempView('sales')
product_df.createOrReplaceTempView('product')

spark.sql('''
with x as(
    select product_id,min(year) as minyear from sales
    group by product_id
)

select product_id,year as first_year,quantity,price
    from sales s
    left join product p using(product_id)
    join x using(product_id)
    where year = minyear
''').show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



In [26]:
spark.stop()